### 기본 세팅

In [1]:
import time
import pandas as pd

from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys

### 유튜브 크롤링

In [2]:
def get_youtube_info(keyword):

    # 드라이버 실행
    driver = webdriver.Chrome('chromedriver.exe')
    driver.implicitly_wait(3)
    sleeptime = 0.5     # 프로세스 일시정지 시간
    
    
    
    # 유튜브 접속
    keyword = keyword + ' 주가'
    driver.get(f'https://www.youtube.com/results?search_query={keyword}')
    time.sleep(sleeptime)

    
    
    # 영상 순서를 '업로드 날짜'로 변경 
    driver.find_element_by_css_selector('#container > ytd-toggle-button-renderer').click()
    driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[5]/ytd-search-filter-renderer[2]/a/div/yt-formatted-string').click()

    
    
    # 스크롤 최대한 내려서 파싱
    body = driver.find_element_by_tag_name('body')
    num  = 0
    while num < 7:
        # 현재 화면길이를 리턴받아 last_height에 넣음
        last_height = driver.execute_script('return document.documentElement.scrollHeight')
        
        for i in range(3):
            # body에 END키 입력해 스크롤내림
            body.send_keys(Keys.END)
            time.sleep(sleeptime)
        num += 1
                
            
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.close()
    
    name      = soup.select('a#video-title')                       # 영상제목 파싱
    video_url = soup.select('a#video-title')                       # 영상주소 파싱
    view      = soup.select('a#video-title')                       # 조회수 파싱
    date      = soup.select('#metadata-line > span.style-scope')     # 게시날짜 파싱

    name_list = []     # 영상제목 리스트
    url_list  = []     # 영상주소 리스트
    view_list = []     # 조회수 리스트
    date_list = []     # 게시날짜 리스트

    
    
    # 영상 제목, 조회수, 게시날짜 크롤링
    for i in range(len(name)):
        name_list.append(name[i].text.strip())                      # 영상 제목 추가
        view_list.append(view[i].get('aria-label').split()[-1])     # 영상 조회수 추가
        
        # 영상 업로드 날짜 처리  
        if '전' in date[1].text:
            date_list.append(date[(2*i)+1].text)
        else : 
            date_list.append(date[2*i+2].text)
    
    # 영상주소 크롤링      
    for i in video_url:
        url_list.append('{}{}'.format('https://www.youtube.com', i.get('href')))
        
        
    
    # 게시날짜 3일전, 2일전, 1일전 영상만 추출
    youtube_info = pd.DataFrame({'제목': name_list, '주소': url_list, '조회수': view_list, '업로드날짜': date_list})
   
    if '3일 전' in date_list:
        final_num = list(youtube_info[youtube_info['업로드날짜'] == '3일 전'].index)[-1]
        youtube_info = youtube_info.iloc[:(final_num+1), :]
    elif '2일 전' in date_list:
        final_num = list(youtube_info[youtube_info['업로드날짜'] == '2일 전'].index)[-1]
        youtube_info = youtube_info.iloc[:(final_num+1), :]
    elif '1일 전' in date_list:
        final_num = list(youtube_info[youtube_info['업로드날짜'] == '1일 전'].index)[-1]
        youtube_info = youtube_info.iloc[:(final_num+1), :]
    else: 
        youtube_info = pd.DataFrame({'제목': [], '주소': [], '조회수': [], '업로드날짜': []})

        
        
    id_final      = []     # 댓글 모을 id 리스트
    comment_final = []     # 주소별 댓글 리스트
    url_final     = []     # 댓글 모을 주소 리스트
    
    # 동영상 주소를 돌며 크롤링
    url_total = list(youtube_info['주소'].values)
    for url in url_total:
        # 드라이버 실행
        driver = webdriver.Chrome('chromedriver.exe')
        driver.get(url)
        time.sleep(sleeptime)

        try:
            driver.find_element_by_css_selector("#dismiss-button > a").click()
        except:
            pass
        
        
        
        # 스크롤 최대한 내려서 파싱
        body = driver.find_element_by_tag_name('body')
        num  = 6
        while num > 0:
            # 현재 화면길이를 리턴받아 last_height에 넣음
            last_height = driver.execute_script('return document.documentElement.scrollHeight')
            
            for i in range(7):
                # body에 END키 입력해 스크롤내림
                body.send_keys(Keys.END)
                time.sleep(sleeptime)
                num -= 1
        time.sleep(sleeptime)
        
        
    
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        id_list      = soup.select("#author-text > span")                  # 댓글 작성자명 파싱
        comment_list = soup.select('yt-formatted-string#content-text')     # 댓글내용 파싱
        num          = len(comment_list)                                   # 영상당 댓글 수
         
        
        
        # 댓글내용, 작성자명 기본 전처리
        for k in range(len(comment_list)):
            url_final.append(url)
            
            temp_id = id_list[k].text
            temp_id = temp_id.replace('\n', '')
            temp_id = temp_id.replace('\t', '')
            temp_id = temp_id.replace('    ', '')
            id_final.append(temp_id)

            temp_comment = comment_list[k].text
            temp_comment = temp_comment.replace('\n', '')
            temp_comment = temp_comment.replace('\t', '')
            temp_comment = temp_comment.replace('    ', '')
            comment_final.append(temp_comment)
        driver.close()
        
                

    youtube_comment = pd.DataFrame({'주소': url_final, '작성자명': id_final, '댓글': comment_final})
    return youtube_info, youtube_comment

### 크롤링 예시

In [3]:
youtube_카카오뱅크_info, youtube_카카오뱅크_comment = get_youtube_info('카카오뱅크')

In [4]:
youtube_카카오뱅크_info

,제목,주소,조회수,업로드날짜
0,카카오뱅크(323410) - 횡보만 조금 더 가져가주면 좋을 것 같습니다만..?,https://www.youtube.com/watch?v=uJNElJ3xD3Q,213회,8시간 전
1,코스피 폭락에도 달리는 ‘카카오뱅크’ 재평가 시작? | [머니방위대 EP.42],https://www.youtube.com/watch?v=GFlVisvnHmo,133회,8시간 전
2,"카카오뱅크·삼성전자 지속 매도 이유는?(전윤호)/ 카카오뱅크, 삼성전자 / 주식경제...",https://www.youtube.com/watch?v=IA6iTqTIhks,"2,122회",10시간 전
3,"당일 관심테마! 개별주,게임,수소,백신,폴더블//카카오뱅크,일진홀딩스,펄어비스,카카...",https://www.youtube.com/watch?v=FEo91IwnlWw,"18,639회",18시간 전
4,[삼성전자 주가] 카카오뱅크 넘어설! 이재용 복귀 특종 이슈! 삼성과 끝까지 함께 ...,https://www.youtube.com/watch?v=7shLZ8cPmzk,980회,1일 전
5,펄어비스 카카오뱅크 sk바이오사이언스 주가전망 매매전략,https://www.youtube.com/watch?v=3TqfL1q_LFA,"1,785회",1일 전
6,[주식]카카오뱅크 꿇을까 못 뚫을까 제대로 돌파합니다 카카오뱅크주가전망,https://www.youtube.com/watch?v=A82bN_Hpnoo,214회,1일 전
7,"SK바이오사이언스, 카카오뱅크 관심종목 주가전망 매매전략 / 시장주도주의 눌림목, ...",https://www.youtube.com/watch?v=w-wgdpaYCTM,"4,480회",1일 전
8,"[삼성전자 주가] 카카오, 카카오뱅크 후속주! 삼성전자 240조 투자 반도체 급등장...",https://www.youtube.com/watch?v=BjQqTGMopJw,733회,1일 전
9,카카오뱅크 주가 전망 및 대응 방안 / 카카오뱅크 주가 전망 / 카카오뱅크 대응 방...,https://www.youtube.com/watch?v=GvXf81MVIbs,677회,1일 전


In [5]:
youtube_카카오뱅크_comment

,주소,작성자명,댓글
0,https://www.youtube.com/watch?v=FEo91IwnlWw,김건우,어려운장 심플님덕에 그나마 덜헤메게됩니다. 정말감사드려요
1,https://www.youtube.com/watch?v=FEo91IwnlWw,스누피,금요일이네요 모듀들 한주 수고 많았습니다 영상 감사합니다🙏🏻🙇‍♀️
2,https://www.youtube.com/watch?v=FEo91IwnlWw,장경덕,항상감사히 듣고 있습니다심플님 성실함과 주린이들챙기는맘재능기부 도움많이 받고있...
3,https://www.youtube.com/watch?v=FEo91IwnlWw,빛초롱,영상 감사합니다~고맙습니다~^^;한주마무리 잘되어 주말 행복하게 보냈음 합니다~ㅎㅎㅎ
4,https://www.youtube.com/watch?v=FEo91IwnlWw,acimsaram,"목소리 좋으시고, 종목 좋으시고! 항상 감사드립니다. 오늘도 성공투자 하세요."
...,...,...,...
109,https://www.youtube.com/watch?v=T3aYYiRJPg4,송영숙,감사합니다
110,https://www.youtube.com/watch?v=T3aYYiRJPg4,임화영 여,정말 짱이에요
111,https://www.youtube.com/watch?v=T3aYYiRJPg4,이성남,네 ᆢ 저도 조금가지고 있는데 쉽게 안오르네요카뱅은 팔았어요
112,https://www.youtube.com/watch?v=qxGi3qG4f2M,엘리 남,전 단기 10만원은 보고 있어서 9만원에 내리기 직전에 구입했네요~ㅎ
